In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import openpyxl
import statistics
import math
import os
from matplotlib.backends.backend_pdf import PdfPages


def diff_x_i(sig, x_i) :
    return [sig[i]-x_i for i in range(len(sig))]


def diff_a_iw_abs(a_i, a_iw) :
    return abs(a_i - a_iw)



    
    
    
#ファイルの読み込み                                                                                                                         
#file_name1 = sys.argv[1]                                                                                                                   
#file_name2 = sys.argv[2]                                                                                                                   
#file1 = pd.ExcelFile(file_name1+'.xlsx')                                                                                                   
#file2 = pd.ExcelFile(file_name2+'.xlsx')                                                                                                   

def write_data(wb_sheet, sig, column_point, start_row, value_title) :
    wb_sheet.cell(column=column_point, row=1, value=value_title)
    for i, sig_data in enumerate(sig) :
        #print(sig_data)
        wb_sheet.cell(column=column_point, row=i+start_row, value=sig_data)

# ディレクトリ作成
def my_makedirs(path):
    if not os.path.isdir(path):
        os.makedirs(path)

In [52]:
def inspect(file1, k, spike_plot=True, type="nomal"):
    #ファイル1のデータカウント                                                                                                              
    pattern_dict1 = {}
    sumpsth = 0
    peak = -0.08
    for i, name in enumerate(sheet_names1):
#        print(wb[name]['volt'])
        sheet_df1[i] = file1.parse(name)
        #print(sheet_df1[i])        
        try :
#            for row in wb[name].columns :
#                sig_ori = []
#                for cell in row : 
                    #print(cell.value)
#                    sig_ori.append(cell.value)
            sig_ori = (sheet_df1[i]["volt"]).values
        except KeyError :
            print("not max data number")
            break
        sig_smooth = [(sig_ori[i]+sig_ori[i+1]+sig_ori[i+2])/3  for i in range(len(sig_ori)-2)]
#        write_data(file1[name], sig_smooth, 2, 3, "smooth")
            
        sig_fire = [ 1 if(sig_smooth[i-3] > sig_smooth[i-2]) & (sig_smooth[i-2] > sig_smooth[i-1]) & (sig_smooth[i-1] > sig_smooth[i]) \
                   & (sig_smooth[i] < sig_smooth[i+1]) & (sig_smooth[i+1] < sig_smooth[i+2]) & (sig_smooth[i+2] < sig_smooth[i+3]) & (sig_ori[i+1] <= peak) \
                    else 0 for i in range(3, len(sig_smooth)-3)]
        
        sig_fire_a = [ 1 if(sig_smooth[i] - sig_smooth[i-1] <= 0) & (sig_smooth[i+1] - sig_smooth[i] >= 0) & (sig_ori[i+1] <= peak) \
                     else 0 for i in range(3, len(sig_smooth) -3)]
        
        
        
        plot_len = len(sig_ori) - (2*k)
        time_plot = [i/25 for i in range(plot_len)]
        plt.figure(figsize=(10, 4))
        
        if(spike_plot == True) :
            plt.subplot(3, 1, 1)
            plt.subplots_adjust(wspace=0.0, hspace=0)
        #plt.figure()
        start_point = int((len(sig_ori)-plot_len)/2)
        if(type=="short") :
            plt.xlim(20, 40)
        else :
            plt.xlim(0, plot_len/25)
        plt.xticks(color="None")
        if(start_point == 0) :
            plt.plot(time_plot, sig_ori, linewidth = 0.5)
        else :
            plt.plot(time_plot, sig_ori[start_point:-start_point], linewidth = 0.5)
        #plt.plot(peak_line)     
        if(spike_plot == True) :       
            for j in range(10) :
                h = 1 + (j/10)
                S_1 = [(max(diff_x_i(sig_ori[i-k:i], sig_ori[i])) + max(diff_x_i(sig_ori[i+1:i+k+1], sig_ori[i])))/2 for i in range(k, len(sig_ori)-k)]
                sig_mean = sum(sig_ori)/len(sig_ori) # m'
                sig_sd = statistics.pstdev(sig_ori) # s'
                S_1_fire = [1 if(S_1[i]-sig_mean > h*sig_sd) & (sig_ori[i+k] < sig_ori[i+k-1]) & (sig_ori[i+k] < sig_ori[i+k+1]) else 0 for i in range(len(S_1))]

                plt.subplot(21, 1, j+8)
                plt.ylim(0.95, 1.05)
                if(type=="short") :
                    plt.xlim(20, 40)
                else :
                    plt.xlim(0, plot_len/25)
                plt.yticks(color="None")
                if(j != 9) :
                    plt.xticks(color="None")
                #plt.scatter([i/25 for i in range(len(sig1))],sig1,marker='s', s=10)
                start_point = int((len(S_1_fire)-plot_len)/2)
                if(start_point == 0) :
                    plt.scatter([i/25 for i in range(len(S_1_fire))], S_1_fire,marker='s', s=3)
                else :
                    plt.scatter([i/25 for i in range(len(S_1_fire[start_point:-start_point]))], S_1_fire[start_point:-start_point],marker='s', s=3)          
        pdf.savefig()
        plt.close()  
        #plt.scatter([i/25 for i in range(len(S_1_fire))], S_1_fire,marker='s', s=10)
        #print(S_1_fire)
        

    


In [55]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/male-25000/B38-HR/B38-male-R2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-data-short/"
save_path = "/home/nodoka/win2/data4.0/send-data/"
ep_individual = "male-25000/B38-HR/"
timing_data = "B38-male-R3"

In [66]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/female-25000/C14-HL/C14-female-L2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-data-short/"
#save_path = "/home/nodoka/win2/data4.0/send-data/"
ep_individual = "female-25000/C14-HL/"
timing_data = "C14-female-L3"

In [93]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/female-25000/B31-HR/B31-female-R2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-data-short/"
#save_path = "/home/nodoka/win2/data4.0/send-data/"
ep_individual = "female-25000/B31-HR/"
timing_data = "B31-female-R3"

In [83]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/restraint-25000/C15-HR/C15-restraint-R2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-data-short/"
#save_path = "/home/nodoka/win2/data4.0/send-data/"
ep_individual = "restraint-25000/C15-HR/"
timing_data = "C15-restraint-R2"

In [91]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/object-25000/B32-HL/B32-object-L2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-data-short/"
save_path = "/home/nodoka/win2/data4.0/send-data/"
ep_individual = "object-25000/B32-HL/"
timing_data = "B32-object-L2"

In [94]:
file1 = pd.ExcelFile(read_path + ep_individual + timing_data + ".xlsx")
#file1 = openpyxl.load_workbook("C14-female-L3.xlsx")
#file1 = pd.ExcelFile("/st9/b009vb/01data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file2 = pd.ExcelFile(sys.argv[2])

sheet_df1 = file1.parse(file1.sheet_names, header=None)
#sheet_df2 = file2.parse(file2.sheet_names, header=None)

#cmap = plt.get_cmap("tab20")                                                                                                               

sheet_names1 = file1.sheet_names
#sheet_names2 = file2.sheet_names

my_makedirs(save_path+ep_individual+timing_data)

for k in range(2, 6) :
    pdf = PdfPages(save_path + ep_individual + timing_data + "/parameta_1h_" + str(k) + "k.pdf")
    inspect(file1, k, True, "short")
    #inspect(file1, k, True)
    pdf.close()